In [1]:
from osgeo import gdal
from osgeo import gdalconst
import os
from osgeo import ogr
from osgeo import osr
import fiona
from ops.ops import load_json
from tqdm.notebook import tqdm_notebook
from osgeo import gdal_array
from skimage.morphology import disk, dilation, erosion
import numpy as np

In [2]:
#image_folder = 'img/GEE-imgs'

paths = load_json(os.path.join('conf', 'paths.json'))
shp = load_json(os.path.join('conf', 'shp.json'))
conf = load_json(os.path.join('conf', 'conf.json'))
shp_path = paths['shp']

img_source = conf['img_source']
grid_shp = shp[f'shp_download_{img_source}']
grid_save = os.path.join('shp', f"{grid_shp}.shp")


In [3]:
def rasterize(img_id):
    image_folder = os.path.join('img', 'GEE_imgs_opt')
    labels_folder = os.path.join('img', 'labels')
    
    image_p = os.path.join(image_folder, f'{img_id}_opt_2019.tif')

    v_def_until_2007 = ogr.Open(os.path.join(shp_path, 'def_2007_pa.shp'))
    l_def_until_2007 = v_def_until_2007.GetLayer()
    
    v_hidro = ogr.Open(os.path.join(shp_path, 'hidro_pa.shp'))
    l_hidro = v_hidro.GetLayer()
    
    v_no_forest = ogr.Open(os.path.join(shp_path, 'no_forest_pa.shp'))
    l_no_forest = v_no_forest.GetLayer()
    
    v_def_2008_2018 = ogr.Open(os.path.join(shp_path, 'def_2008_2018_pa.shp'))
    l_def_2008_2018 = v_def_2008_2018.GetLayer()
    
    v_def_2019 = ogr.Open(os.path.join(shp_path, 'def_2019_pa.shp'))
    l_def_2019 = v_def_2019.GetLayer()
    
    v_def_2020 = ogr.Open(os.path.join(shp_path, 'def_2020_pa.shp'))
    l_def_2020 = v_def_2020.GetLayer()
    
    v_def_2020_l = ogr.Open(os.path.join(shp_path, 'def_2020_l_pa.shp'))
    l_def_2020_l = v_def_2020_l.GetLayer()
    
    data = gdal.Open(image_p, gdalconst.GA_ReadOnly)
    
    geo_transform = data.GetGeoTransform()
    x_min = geo_transform[0]
    y_max = geo_transform[3]
    x_max = x_min + geo_transform[1] * data.RasterXSize
    y_min = y_max + geo_transform[5] * data.RasterYSize
    x_res = data.RasterXSize
    y_res = data.RasterYSize
    
    crs = data.GetSpatialRef()
    proj = data.GetProjection()

    pixel_width = geo_transform[1]
    output = os.path.join(labels_folder, f'label_{img_id}.tif')
    target_ds = gdal.GetDriverByName('GTiff').Create(output, x_res, y_res, 1, gdal.GDT_Byte)
    #target_ds.SetGeoTransform((x_min, geo_transform[1], 0, y_min, 0, geo_transform[5]))
    target_ds.SetGeoTransform(geo_transform)
    target_ds.SetSpatialRef(crs)
    target_ds.SetProjection(proj)
    band = target_ds.GetRasterBand(1)
    #NoData_value = 0
    #band.SetNoDataValue(NoData_value)
    band.FlushCache()
    
    gdal.RasterizeLayer(target_ds, [1], l_def_until_2007, burn_values=[2])
    gdal.RasterizeLayer(target_ds, [1], l_hidro, burn_values=[2])
    gdal.RasterizeLayer(target_ds, [1], l_no_forest, burn_values=[2])
    gdal.RasterizeLayer(target_ds, [1], l_def_2008_2018, burn_values=[2])
    gdal.RasterizeLayer(target_ds, [1], l_def_2019, burn_values=[2])
    
    gdal.RasterizeLayer(target_ds, [1], l_def_2020, burn_values=[1])
    rasterized_data = target_ds.ReadAsArray() 
    defor_data = rasterized_data == 1
    defor_data = defor_data.astype(np.uint8)

    border_data = dilation(defor_data, disk(1)) - erosion(defor_data, disk(1))
    rasterized_data[border_data==1] = 2

    target_ds.GetRasterBand(1).WriteArray(rasterized_data)

    #gdal.RasterizeLayer(target_ds, [1], l_def_2020_l, burn_values=[2], options = ['ALL_TOUCHED=TRUE'])
    
    #print(target_ds.GetGeoTransform())
    target_ds = None
    
    
    

In [4]:
with fiona.open(grid_save) as grid:
    for feat in tqdm_notebook(grid):
        rasterize(int(feat['properties']['id']))
        #print(feat['properties']['id'])

  0%|          | 0/30 [00:00<?, ?it/s]